In [2]:
import numpy as np

In [1]:
Id = lambda x, *vargs, **kvargs: x
MESSAGE = "Starting point outside of the domain."

In [31]:
def extract_options(options, prefix):
    """extract_options(dict(law=0, law_a=1, law_b=2, foo=3, foo_c=4), 'law') == {'a': 1, 'b': 2}"""
    return {k.replace(prefix+'_', ""):options[k] for k in options if k.find(prefix+'_')==0}

In [3]:
def incube(x, cube):  # whether in the open cube
    x = np.array(x)
    cube = np.array(cube)
    return np.alltrue(cube[:, 0] < x) and np.alltrue(x < cube[:, 1])

In [19]:
def infinite_cube(d):  # make d-dimensional infinite cube
    return np.tile([-np.inf, np.inf], (d, 1))

In [55]:
def uniform(x, width=1, *vargs, seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    if np.isscalar(width):
        width = width * np.ones(d)
    if seed is not None:
        np.random.seed(seed)
        
    return x + width * (np.random.rand(d) - 0.5)


def gaussian(x, sigma=1, *vargs, seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    if np.isscalar(sigma):
        sigma = sigma * np.ones(d)
    if seed is not None:
        np.random.seed(seed)
    
    return x + sigma * (np.random.randn(d))

In [72]:
def individual(x, cube=None, law=uniform, *vargs, **options):
    x = np.array(x)
    d = len(x)

    if cube is None:
        cube = infinite_cube(d)
    cube = np.array(cube)
    
    if not incube(x, cube):
        raise Exception(MESSAGE)
        
    y = law(x, **extract_options(options, 'law'))
    index = (y < cube[:, 0]) | (y > cube[:, 1])
    y[index] = x[index]
    return y


def collective(x, cube=None, cov=1, *vargs, seed=None, **kvargs):  # multivariate-normal distribution
    x = np.array(x)
    d = len(x)

    if cube is None:
        cube = infinite_cube(d)
    cube = np.array(cube)
    if not incube(x, cube):
        raise Exception(MESSAGE)
    
    if np.isscalar(cov):
        cov = cov * np.ones(d)
    cov = np.array(cov)
    if cov.ndim == 1:
        cov = np.diag(cov)

    if seed is not None:
        np.random.seed(seed)    

    y = x + np.random.multivariate_normal(np.zeros(d), cov)

    return y if incube(y, cube) else x


def rotative(x, t=0, state=None, cube=None, law=uniform, *vargs, seed=None, **options):  
    x = np.array(x)
    d = len(x)

    if seed is not None and t==0:
        np.random.seed(seed)
        
    if cube is None:
        cube = infinite_cube(d)
    cube = np.array(cube)
    
    if not incube(x, cube):
        raise Exception(MESSAGE)
    
    r = t % d
    y = law([x[r]], **extract_options(options, 'law'))
    if cube[r, 0] < y < cube[r, 1]:
        x[r] = y
        
    return x, state

In [59]:
def mh(x, proba, domain=None, N=500, B=200, move='n', ascdes=(Id, Id), seed=None, **kvargs):
    x = np.array(x)
    d = len(x)
    
    if domain is None:
        domain = np.ones((d, 2))
        domain[:, 0] = -np.inf * domain[:, 0]
        domain[:, 1] = np.inf * domain[:, 1]
    domain = np.array(domain)
    
    if not indomain(x, domain):
        raise Exception(MESSAGE)
    
    if seed is not None:
        np.random.seed(seed)
        
    walker = np.zeros((N+1, d))
    walker[0, :] = x
    
    px = proba(x)

    _x = ascdes[0](x)
    _domain = ascdes[0](domain)
    
    for i in range(N):
        if move in ['u', 'n']:
            _y = move_elementwise(_x, move, _domain, **kvargs)
        elif move == 'c':
            _y = move_cohert(_x, _domain, **kvargs)
        else:
            _y = move(_x, _domain, **kvargs)
        
        y = ascdes[1](_y)
        py = proba(y)
        if np.random.rand() < py / px:
            _x, x, px = _y, y, py
        
        walker[i+1, :] = x
        
    return np.mean(walker[B:, :], axis=0), walker